## <font color='darkblue'>Preface</font>
([article source](https://machinelearningmastery.com/simple-genetic-algorithm-from-scratch-in-python/)) <font size='3ptx'>**The [genetic algorithm](https://en.wikipedia.org/wiki/Genetic_algorithm) is a stochastic global optimization algorithm.**</font>

It may be one of the most popular and widely known biologically inspired algorithms, along with artificial neural networks.

The algorithm is a type of evolutionary algorithm and performs an optimization procedure inspired by the biological theory of evolution by means of natural selection with a binary representation and simple operators based on genetic recombination and genetic mutations.

**In this tutorial, you will discover the genetic algorithm optimization algorithm.** After completing this tutorial, you will know:
* Genetic algorithm is a stochastic optimization algorithm inspired by evolution.
* How to implement the genetic algorithm from scratch in Python.
* How to apply the genetic algorithm to a continuous objective function.


### <font color='darkgreen'>Tutorial Overview</font>
This tutorial is divided into four parts; they are:
* <font size='3ptx'>[**Genetic Algorithm**](#sect1)</font>
* <font size='3ptx'>[**Genetic Algorithm From Scratch**](#sect2)</font>
* <font size='3ptx'>[**Genetic Algorithm for OneMax**](#sect3)</font>
* <font size='3ptx'>[**Genetic Algorithm for Continuous Function Optimization**](#sect4)</font>

<a id='sect1'></a>
## <font color='darkblue'>Genetic Algorithm</font>
**The [Genetic Algorithm](https://en.wikipedia.org/wiki/Genetic_algorithm) is a stochastic global search optimization algorithm.**

**It is inspired by the biological theory of evolution by means of natural selection**. Specifically, the new synthesis that combines an understanding of genetics with the theory.
> Genetic algorithms (algorithm 9.4) borrow inspiration from biological evolution, where fitter individuals are more likely to pass on their genes to the next generation.<br/><br/>
> [— Page 148, Algorithms for Optimization, 2019.](https://amzn.to/2Traqek)
<br/>

The algorithm uses analogs of a **genetic representation** (<font color='brown'>bitstrings</font>), **fitness** (<font color='brown'>function evaluations</font>), **genetic recombination** (<font color='brown'>crossover of bitstrings</font>), and **mutation** (<font color='brown'>flipping bits</font>).

The algorithm works by first creating a population of a fixed size of random bitstrings. The main loop of the algorithm is repeated for a fixed number of iterations or until no further improvement is seen in the best solution over a given number of iterations.

**One iteration of the algorithm is like an evolutionary generation.**

First, **the population of bitstrings** (<font color='brown'>candidate solutions</font>) **are evaluated using the <font color='darkblue'>objective function</font>**. The objective function evaluation for each candidate solution is taken as the fitness of the solution, which may be minimized or maximized.

Then, parents are selected based on their fitness. A given candidate solution may be used as parent zero or more times. **A simple and effective approach to selection involves drawing `k` candidates from the population randomly and selecting the member from the group with the best fitness. This is called <font color='darkblue'>tournament selection</font> where `k` is a hyperparameter and set to a value such as 3**. This simple approach simulates a more costly fitness-proportionate selection scheme.
> In tournament selection, each parent is the fittest out of <i>k</i> randomly chosen chromosomes of the population <br/><br/>
> [— Page 151, Algorithms for Optimization, 2019.](https://amzn.to/2Traqek)
<br/>

**Parents are used as the basis for generating the next generation of candidate points and one parent for each position in the population is required.**

Parents are then taken in pairs and used to create two children. <font color='darkblue'>Recombination</font> is performed using a crossover operator. This involves selecting a random split point on the bit string, then creating a child with the bits up to the split point from the first parent and from the split point to the end of the string from the second parent. This process is then inverted for the second child.

For example the two parents:
* parent1 = 00000
* parent2 = 11111

May result in two cross-over children:
* child1 = 00011
* child2 = 11100

**This is called <font color='darkblue'>one point crossover</font>, and there are many other variations of the operator.**

**Crossover is applied probabilistically for each pair of parents, meaning that in some cases, copies of the parents are taken as the children instead of the recombination operator**. Crossover is controlled by a hyperparameter set to a large value, such as 80 percent or 90 percent.
> Crossover is the Genetic Algorithm’s distinguishing feature. It involves mixing and matching parts of two parents to form children. How you do that mixing and matching depends on the representation of the individuals.<br/><br/>
> [— Page 36, Essentials of Metaheuristics, 2011.](https://amzn.to/2HxZVn4)
<br/>

**<font color='darkblue'>Mutation</font> involves flipping bits in created children candidate solutions**. Typically, the mutation rate is set to `1/L`, where `L` is the length of the bitstring.

> Each bit in a binary-valued chromosome typically has a small probability of being flipped. For a chromosome with `m` bits, this mutation rate is typically set to `1/m`, yielding an average of one mutation per child chromosome. <br/><br/>
> [— Page 155, Algorithms for Optimization, 2019.](https://amzn.to/2Traqek)
<br/>

For example, if a problem used a bitstring with 20 bits, then a good default mutation rate would be (1/20) = 0.05 or a probability of 5 percent.

This defines the simple genetic algorithm procedure. It is a large field of study, and there are many extensions to the algorithm.

Now that we are familiar with the simple genetic algorithm procedure, let’s look at how we might implement it from scratch.

<a id='sect2'></a>
## <font color='darkblue'>Genetic Algorithm From Scratch</font>
**In this section, we will develop an implementation of the genetic algorithm.**

The first step is to create a population of random bitstrings. We could use boolean values True and False, string values ‘0’ and ‘1’, or integer values 0 and 1. In this case, we will use integer values.

We can generate an array of integer values in a range using the [randint()](https://numpy.org/doc/stable/reference/random/generated/numpy.random.randint.html) function, and we can specify the range as values starting at 0 and less than 2, e.g. 0 or 1. We will also represent a candidate solution as a list instead of a NumPy array to keep things simple.
```python
# initial population of random bitstring
pop = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
```

An initial population of random bitstring can be created as follows, where **“`n_pop`” is a hyperparameter that controls the population size and “`n_bits`” is a hyperparameter that defines the number of bits in a single candidate solution**:

In [1]:
from numpy.random import randint
from numpy.random import rand

In [8]:
# bits
n_bits = 20
# define the population size
n_pop = 100
# Population
pop = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
pop[:10]
# Crossover rate
r_cross = 0.9

Next, we can enumerate over a fixed number of algorithm iterations, in this case, controlled by a hyperparameter named “`n_iter`“.

In [3]:
# define the total iterations
n_iter = 100

The first step in the algorithm iteration is to evaluate all candidate solutions. **We will use a function named <font color='blue'>objective()</font> as a generic objective function and call it to get a fitness score, which we will minimize**.
```python
# evaluate all candidates in the population
scores = [objective(c) for c in pop]
```

We can then select parents that will be used to create children.

**The tournament selection procedure can be implemented as a function that takes the population and returns one selected parent.** The `k` value is fixed at 3 with a default argument, but you can experiment with different values if you like:

In [4]:
# tournament selection
def selection(pop, scores, k=3):
    # first random selection
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), k-1):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

We can then call this function one time for each position in the population to create a list of parents:
```python
# select parents
selected = [selection(pop, scores) for _ in range(n_pop)]
```
We can then create the next generation.

This first requires a function to perform crossover. This function will take two parents and the crossover rate. The crossover rate is a hyperparameter that determines whether crossover is performed or not, and if not, the parents are copied into the next generation. It is a probability and typically has a large value close to 1.0.

The <font color='blue'>crossover()</font> function below implements crossover using a draw of a random number in the range `[0,1]` to determine if crossover is performed, then selecting a valid split point if crossover is to be performed.

In [5]:
# crossover two parents to create two children
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if rand() < r_cross:
        # select crossover point that is not on the end of the string
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
        
    return [c1, c2]

We also need a function to perform mutation.

This procedure simply flips bits with a low probability controlled by the “`r_mut`” hyperparameter.

In [6]:
# mutation rate
r_mut = 1.0 / float(n_bits)

# mutation operator
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]

**We can then loop over the list of parents and create a list of children to be used as the next generation**, calling the <font color='blue'>crossover</font> and <font color='blue'>mutation</font> functions as needed. Below is sample code:
```python
children = []
for i in range(0, n_pop, 2):
    # Get selected parent in pairs
    p1, p2 = selected[i], selected[i+1]
    # Crossover & Mutation operation
    for c in crossover(p1, p2, r_cross):       
        mutation(c, r_mut)
        # Store for next generation
        children.append(c)
```
We can tie all of this together into a function named <font color='blue'>genetic_algorithm()</font> that takes the name of the objective function and the hyperparameters of the search, and returns the best solution found during the search.

In [15]:
# Genetic algorithm
def genetic_algorithm(objective, n_bits=n_bits, n_pop=n_pop, r_cross=r_cross, r_mut=r_mut):
    # Initialize population of random bitstring
    pop = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
    # Keep track of best solution
    best, best_eval = 0, objective(pop[0])
    # Enumerate generations
    for gen in range(n_iter):
        # Evaluate all candidates in the population
        scores = [objective(c) for c in pop]
        # Check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(f"> {gen}, new best f({pop[i]}) = {scores[i]:.03f}")
                
        # Select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]
        # Create the next generation
        children = []
        for i in range(0, n_pop, 2):
            p1, p2 = selected[i], selected[i+1]
            # Crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # Mutation
                mutation(c, r_mut)
                # Store for next generation
                children.append(c)
                    
        # Replace population
        pop = children
            
    return [best, best_eval]

Now that we have developed an implementation of the genetic algorithm, let’s explore how we might apply it to an objective function.

<a id='sect3'></a>
## <font color='darkblue'>Genetic Algorithm for OneMax</font>
**In this section, we will apply the genetic algorithm to a binary string-based optimization problem.**

The problem is called OneMax and evaluates a binary string based on the number of 1s in the string. For example, a bitstring with a length of 20 bits will have a score of 20 for a string of all 1s.

Given we have implemented the genetic algorithm to minimize the objective function, we can add a negative sign to this evaluation so that large positive values become large negative values.

The <font color='blue'>onemax()</font> function below implements this and takes a bitstring of integer values as input and returns the negative sum of the values.

In [10]:
# Objective function
def onemax(x):
    return -sum(x)

Next, we can configure the search.

The search will run for `100` iterations and we will use `20` bits in our candidate solutions, meaning the optimal fitness will be `-20.0`. The population size will be 100, and we will use a crossover rate of 90 percent and a mutation rate of 5 percent. This configuration was chosen after a little trial and error.

In [11]:
# define the total iterations
n_iter = 100
# bits
n_bits = 20
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / float(n_bits)

The search can then be called and the best result reported:

In [16]:
# Perform the GE search
best, score = genetic_algorithm(onemax, n_bits, n_iter, n_pop)
print("Done!")
print(f"f({best}) = {score:.03f}")

> 0, new best f([1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1]) = -14.000
> 0, new best f([0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1]) = -16.000
> 2, new best f([1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0]) = -17.000
> 3, new best f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1]) = -18.000
> 4, new best f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1]) = -19.000
> 9, new best f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -20.000
Done!
f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -20.000


Running the example will report the best result as it is found along the way, **then the final best solution at the end of the search, which we would expect to be the optimal solution**.

<a id='sect4'></a>
## <font color='darkblue'>Genetic Algorithm for Continuous Function Optimization</font>
**Optimizing the OneMax function is not very interesting; we are more likely to want to optimize a continuous function.**

For example, we can define the $x^2$ minimization function that takes input variables and has an optima at $f(0, 0) = 0.0$.

In [17]:
# Objective function
def objective(x):
    return x[0]**2.0 + x[1]**2.0

We can minimize this function with a genetic algorithm.

First, we must define the bounds of each input variable.

In [18]:
# Define the range for input
bounds = [[-5.0, 5.0], [-5.0, 5.0]]

We will take the `n_bits` hyperparameter as a number of bits per input variable to the objective function and set it to 16 bits.

In [19]:
# bit per variable
n_bits = 16

This means our actual bit string will have `(16*2) = 32` bits, given the two input variables. We must update our mutation rate accordingly:

In [20]:
# Mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))

Next, we need to ensure that the initial population creates random random bitstrings that are large enough:
```python
pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
```

Finally, we need to decide the bitstrings to numbers prior to evaluating each with the object function.

We can achieve this by first decoding each substring to an integer, then scaling the integer to the desired range. This will give a vector of values in the range that can then be provided to the objective function for evaluation.

The <font color='blue'>decode()</font> function below implements this, taking the bounds of the function, the number of bits per variable, and a bitstring as input and returns a list of decoded real values:

In [27]:
# Decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
    decoded = []
    largest = 2 ** n_bits
    for i in range(len(bounds)):
        start, end = i * n_bits, (i * n_bits) + n_bits
        substring = bitstring[start:end]
        # Covert bitstring to a string of chars
        chars = ''.join([str(b) for b in substring])
        # Covert string to integer
        int_value = int(chars, 2)
        # Scale integer to desired range
        nrm_value = bounds[i][0] + (int_value/largest) * (bounds[i][1] - bounds[i][0])
        # Store
        decoded.append(nrm_value)
    
    return decoded

We can then call this at the beginning of the algorithm loop to decode the population, then evaluate the decoded version of the population.
```python
# Eecode population
decoded_pop = [decode(bounds, n_bits, p) for p in pop]
# Evaluate all candidates in the population
scores = [objective(d) for d in decoded_pop]
```

Trying this together, the complete example of the GE for continuous function optimization is listed below:

In [28]:
def genetic_algorithm_4_cont(objective, bounds=bounds, decode_func=decode, n_bits=n_bits, n_iter=n_iter, n_pop=n_pop, r_cross=r_cross, r_mut=r_mut):
    # Initialize population of random bitstring
    pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
    decoded_pop = [decode_func(bounds, n_bits, p) for p in pop]
    # Keep track of best solution
    best, best_eval = 0, objective(decoded_pop[0])
    # Enumerate generations
    for gen in range(n_iter):
        # Evaluate all candidates in the population
        scores = [objective(c) for c in decoded_pop]
        # Check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(f"> {gen}, new best f({pop[i]}) = {scores[i]:.03f}")
                
        # Select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]
        # Create the next generation
        children = []
        for i in range(0, n_pop, 2):
            p1, p2 = selected[i], selected[i+1]
            # Crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # Mutation
                mutation(c, r_mut)
                # Store for next generation
                children.append(c)
                    
        # Replace population
        pop = children
        decoded_pop = [decode_func(bounds, n_bits, p) for p in pop]
            
    return [best, best_eval]

In [29]:
best, score = genetic_algorithm_4_cont(objective)
print("Done!")
decoded = decode(bounds, n_bits, best)
print(f"({decoded}) = {score:.03f}")

> 0, new best f([1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0]) = 0.659
> 0, new best f([1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1]) = 0.475
> 0, new best f([0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1]) = 0.359
> 1, new best f([0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]) = 0.307
> 1, new best f([0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]) = 0.143
> 1, new best f([0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1]) = 0.109
> 1, new best f([0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1]) = 0.031
> 2, new best f([0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1]) = 0.027
> 3, new best f([0, 1, 1

Running the example reports the best decoded results along the way and the best decoded solution at the end of the run. n this case, we can see that the algorithm discovers an input very close to $f(0.0, 0.0) = 0.0$.